## 1. BASE PROGRAMA

#### 1.1 IMPORTACION PAQUETES

In [ ]:
import time
from pynq import GPIO
import paho.mqtt.client as mqtt
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import ArduinoSEEEDGroveAdapter
from time import sleep
import sys
import time
import json
import time

#### 1.2 DEFINICION SENSORES Y ACTUADORES

In [ ]:
base = BaseOverlay("base.bit")

In [ ]:
adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, A0='grove_temperature', I2C='grove_oled', 
                                   D4='grove_buzzer', D5='grove_led_stick', D6 = 'grove_line_finder', D7 = 'grove_relay')

In [ ]:
sensor_temp = adapter.A0
oled = adapter.I2C
actuador_buzzer = adapter.D4
led = adapter.D5
sensor_particulas = adapter.D6
rele = adapter.D7

#### 1.3 DEFINICION FUNCIONES AUXILIARES

In [ ]:
#FUNCION QUE DEVUELVE EL COLOR EN HEXADECIMAL SEGUN VALOR.
def get_color(value):
    # Limitar el valor entre -10 y 10
    value = max(-10, min(value, 10))
    
    if value < 0:
        # Escala de -10 a 0: azul (0x0000FF) a verde (0x00FF00)
        ratio = (value + 10) / 10  # Normaliza a [0, 1]
        red = 0
        green = int(255 * ratio)   # Aumenta de 0 a 255
        blue = int(255 * (1 - ratio))  # Decrece de 255 a 0
    else:
        # Escala de 0 a 10: verde (0x00FF00) a rojo (0xFF0000)
        ratio = value / 10         # Normaliza a [0, 1]
        red = int(255 * ratio)     # Aumenta de 0 a 255
        green = int(255 * (1 - ratio))  # Decrece de 255 a 0
        blue = 0
    
    # Generar el color en formato 0xRRGGBB
    return int(f"0x{red:02X}{green:02X}{blue:02X}", 16)

#FUNCION QUE MODIFICA EL MENSAJE DE LA PANTALLA OLED.
def actualizar_pantalla(oled, estado, temp, dif, tideal, incendio):
    oled.clear_display()  # Limpiar la pantalla antes de cada actualización

    if estado == "normal":
        # Modo normal: Mostrar temperatura actual e ideal
        oled.set_position(0, 0)
        oled.put_string(f"Temp: {temp} C")
        oled.set_position(1, 0)
        oled.put_string(f"Ideal: {tideal} C")
        oled.set_normal_display()
        oled.set_contrast_level(150)  # Contraste moderado

    elif estado == "ajuste":
        # Modo ajuste: Mostrar la diferencia y ajustar ventilador
        oled.set_position(0, 0)
        oled.put_string("Ajustando...")
        oled.set_position(1, 0)
        oled.put_string(f"Dif: {dif:+} C")
        oled.set_normal_display()
        oled.set_contrast_level(200)  # Contraste alto

    elif estado == "emergencia":
        # Modo emergencia: Parpadeo y alerta de incendio
        for _ in range(5):  # Parpadea 5 veces
            oled.clear_display()
            time.sleep(0.3)  # Pantalla apagada por 0.3 segundos
            oled.set_position(0, 0)
            oled.put_string("!!! ALERTA !!!")
            oled.set_position(1, 0)
            oled.put_string("INCENDIO!!!")
            oled.set_inverse_display()  # Modo inverso para destacar
            oled.set_contrast_level(255)  # Contraste máximo
            time.sleep(0.3)  # Pantalla encendida por 0.3 segundos

        # Detener parpadeo y mantener el mensaje
        oled.clear_display()
        oled.set_position(0, 0)
        oled.put_string("!!! ALERTA !!!")
        oled.set_position(1, 0)
        oled.put_string("Evacuar ahora")
        oled.set_inverse_display()
    
    if estado == "frio":
        # Modo frio: Mostrar temperatura actual e ideal
        oled.set_position(0, 0)
        oled.put_string(f"Temp: {temp} C")
        oled.set_position(1, 0)
        oled.put_string(f"Ideal: {tideal} C")
        oled.set_normal_display()
        oled.set_contrast_level(150)  # Contraste moderado
        

def recoger_datos(tiempo = 1):
    """
    Recopila los datos de los sensores.
    """
    part = 0
    start = time.time()
    while time.time() - start < tiempo:
        part += int(sensor_particulas.line_found())
        
    temp = sensor_temp.get_temperature()
    
    return temp, part


def sonido_luces_alarma(duracion, delay_luces=10):
    """
    Ejecuta sonido y luces como alarmas combinadas durante un tiempo especificado.

    Args:
        duracion: Duración total en segundos para ejecutar la alarma.
        delay_luces: Número de ciclos de sonido que deben pasar antes de mover las luces.
    """
    # Configuración del sonido
    mini = 500
    maxi = 2000
    interval = 10
    tone = mini
    direc_tone = 1

    # Configuración de las luces
    led.clear()
    pos = 0
    direc_led = 1
    contador_luces = 0  # Contador para ralentizar las luces
    ciclos_completos = 0  # Contador de ciclos completos izquierda-derecha

    # Temporizador
    tiempo_inicio = time.time()

    while time.time() - tiempo_inicio < duracion:
        # Parpadeo de las luces cada 3 ciclos completos
        if ciclos_completos % 5 == 0 and ciclos_completos > 0:
            for _ in range(3):  # Parpadeo tres veces
                for i in range(10):
                    led.set_pixel(i, 0xFF0000)  # Rojo
                led.show()
                time.sleep(0.2)  # Tiempo de luz encendida
                led.clear()
                time.sleep(0.2)  # Tiempo de luz apagada
            ciclos_completos = 0  # Reiniciar el contador de ciclos completos

        # Sonido del buzzer
        else:
            if direc_tone:
                actuador_buzzer.play_tone(tone, 10)
                if tone == maxi:
                    tone -= interval
                    direc_tone = 0
                else:
                    tone += interval
            else:
                actuador_buzzer.play_tone(tone, 10)
                if tone == mini:
                    tone += interval
                    direc_tone = 1
                else:
                    tone -= interval

            # Movimiento de las luces
            if contador_luces >= delay_luces:
                led.set_pixel(pos, 0xFF0000)  # Rojo
                led.set_pixel(pos + 1, 0xFF0000)
                led.set_pixel(pos + 2, 0xFF0000)
                led.show()
                led.clear()

                if direc_led:
                    if pos == 7:
                        direc_led = 0
                        pos -= 1
                        ciclos_completos += 1  # Completa un ciclo izquierda-derecha
                    else:
                        pos += 1
                else:
                    if pos == 0:
                        direc_led = 1
                        pos += 1
                    else:
                        pos -= 1

                # Reiniciar el contador de luces
                contador_luces = 0
            else:
                contador_luces += 1

##### 1.3.1 Definicion funciones mqtt

In [ ]:
class MQTTPynq:
    """
    Clase para manejar la lógica MQTT en la PYNQ.
    Se conecta a un broker local (127.0.0.1) y:
      - Publica los datos de sensores en 'datos_sensores'
      - Escucha los resultados en 'datos_modelos'
    """
    
    def __init__(self, broker='127.0.0.1', port=1883):
        self.broker = broker
        self.port = port
        
        # Tópicos
        self.TOPIC_SENS = "datos_sensores"
        self.TOPIC_MODEL = "datos_modelos"
        
        # Estado interno
        self.connected_flag = False
        self.respuesta_recibida = False
        
        # Atributos para almacenar la respuesta
        self.tideal = None
        self.incendio = None
        self.iteracion = None
        
        # Crear cliente MQTT
        self.client = mqtt.Client()
        
        # Asignar callbacks
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc):
        """
        Se llama cuando la PYNQ se conecta al broker local.
        """
        if rc == 0:
            self.connected_flag = True
            print("[PYNQ] Conectado al broker (127.0.0.1), rc=", rc)
            self.client.subscribe(self.TOPIC_MODEL)
        else:
            print("[PYNQ] Error al conectar. RC=", rc)
            sys.exit(-1)

    def on_message(self, client, userdata, msg):
        """
        Se llama cuando la PYNQ recibe un mensaje en 'datos_modelos'.
        """
        print(f"[PYNQ] Mensaje en {msg.topic}: {msg.payload.decode()}")
        try:
            datos_respuesta = json.loads(msg.payload.decode())
            print(f"[PYNQ] Respuesta procesada: {datos_respuesta}")

            # Asignar los valores del modelo a los atributos
            self.tideal = datos_respuesta.get("tideal", 0)
            self.incendio = datos_respuesta.get("incendio", False)
            self.iteracion = datos_respuesta.get("iteracion", 0)
            
            # Cuando recibimos el resultado, marcamos que el ciclo ha terminado
            self.respuesta_recibida = True

        except json.JSONDecodeError:
            print("[PYNQ] Error decodificando JSON.")

    def conectar(self):
        """
        Conecta al broker y arranca el loop en segundo plano.
        """
        try:
            self.client.connect(self.broker, self.port, 60)
        except Exception as e:
            print("[PYNQ] Error de conexión:", e)
            sys.exit(-1)
        
        self.client.loop_start()
        
        # Esperar a que esté conectado
        while not self.connected_flag:
            time.sleep(0.1)

    def publicar_sensores(self, temp, part, iteracion):
        """
        Publica datos de sensores en el tópico 'datos_sensores'.
        """
        datos_sensores = {
        "temp": temp,
        "part": part,
        "iteracion": iteracion
        }
        self.client.publish(self.TOPIC_SENS, json.dumps(datos_sensores))
        print(f"[PYNQ] Publicado en '{self.TOPIC_SENS}': {datos_sensores}")

    def esperar_respuesta(self):
        """
        Espera hasta que se reciba la respuesta (o Ctrl+C).
        """
        try:
            while not self.respuesta_recibida:
                time.sleep(0.1)
        except KeyboardInterrupt:
            print("[PYNQ] Interrupción manual...")

    def cerrar(self):
        """
        Detiene el loop y desconecta.
        """
        print("[PYNQ] Ciclo completado. Terminando ejecución...")
        self.client.loop_stop()
        self.client.disconnect()

    def run(self, temp, part,iteracion):
        """
        Método para publicar datos y esperar respuesta.
        """
        # Publicar datos
        self.respuesta_recibida = False
        self.publicar_sensores(temp, part, iteracion)
        print(f"[PYNQ] Esperando respuesta para iteración {iteracion}...")

        # Esperar respuesta
        self.esperar_respuesta()

        # Devolver los valores recibidos
        return  self.tideal, self.incendio, self.iteracion
    

class MQTTAlexa:
    """
    Clase para manejar la lógica MQTT para enviar mensajes a Alexa.
    """
    def __init__(self, broker='127.0.0.1', port=1883):
        self.broker = broker
        self.port = port
        self.TOPICO_ALEXA = "PYNQ-Z2/alexa"
        
        # Estado interno
        self.connected_flag = False
        
        # Crear cliente MQTT
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect

    def on_connect(self, client, userdata, flags, rc):
        """
        Callback al conectar al broker.
        """
        if rc == 0:
            self.connected_flag = True
            print("[ALEXA] Conectado al broker MQTT correctamente.")
        else:
            print(f"[ALEXA] Error al conectar con el broker. Código RC={rc}")

    def conectar(self):
        """
        Conecta al broker y arranca el loop en segundo plano.
        """
        try:
            print(f"[ALEXA] Intentando conectar al broker MQTT en {self.broker}:{self.port}...")
            self.client.connect(self.broker, self.port, 60)
            self.client.loop_start()

            # Esperar a que el cliente se conecte
            max_retries = 10
            retries = 0
            while not self.connected_flag and retries < max_retries:
                print("[ALEXA] Esperando conexión al broker...")
                time.sleep(1)
                retries += 1

            if not self.connected_flag:
                print("[ERROR] No se pudo conectar al broker MQTT después de varios intentos.")
            else:
                print("[ALEXA] Cliente MQTT conectado correctamente.")

        except Exception as e:
            print(f"[ERROR] Error al intentar conectar con el broker: {e}")

    def enviar_mensaje(self, mensaje):
        """
        Publica un mensaje en el tópico de Alexa.

        Args:
            mensaje: Texto que deseas enviar para que Alexa lo reproduzca.
        """
        try:
            if self.connected_flag:
                # Crear el mensaje en formato JSON
                payload = mensaje
                # Publicar el mensaje en el tópico
                resultado = self.client.publish(self.TOPICO_ALEXA, payload)
                if resultado[0] == 0:
                    print(f"[ALEXA] Mensaje enviado correctamente: {payload}")
                else:
                    print(f"[ERROR] Publicación fallida. Código: {resultado[0]}")
            else:
                print("[ERROR] Cliente MQTT no conectado. No se puede enviar mensaje a Alexa.")
        except Exception as e:
            print(f"[ERROR] Error al enviar mensaje a Alexa: {e}")

    def cerrar(self):
        """
        Detiene el loop y desconecta del broker.
        """
        print("[ALEXA] Desconectando del broker MQTT...")
        self.client.loop_stop()
        self.client.disconnect()


#### 1.4 DEFINICION PROGRAMA PRINCIPAL

In [ ]:
pynq_client = MQTTPynq()
pynq_client.conectar()  
alexa_client = MQTTAlexa()
alexa_client.conectar()
iteracion = 1
estado = 0
while base.switches[0].read()==0: # MIENTRAS EL SWITCH 0 NO ESTE ACCIONADO.
    print("-------------------------------------------------------------------------------------")
    print(f"Iteración de lectura y envio de datos número: {iteracion}")
    temp, part = recoger_datos() #Leemos datos de los sensores.
    temp -=  24 #Ajustamos la temperatura par que quede estandarizada (al ponerlo en 5V el sensor de temperatura suma mas o menos 24 grados)
    
    tideal, incendio,ite  = pynq_client.run(temp, part, iteracion) #ENVIO/RECEPCION DE INFORMACION VIA MQTT A EL MODELO:
    print(f"Valores Enviados de los Sensores: Temperatura={temp}, Particulas={part}")
    print(f"Valores Recibidos por el Modelo: tideal={tideal}, incendio={incendio}, iteración{ite}")
    dif = temp - tideal
    
    #TOMA DE DECISIONES PROGRAMA (EJECUCIÓN ACTUADORES) --------------------------------------------------------------:
    if not incendio:
        estado = 1
        col = get_color(dif)
        #LED.
        led.clear()
        for i in range(10):
            led.set_pixel(i, col)
        led.show()
          
        if dif > -4 and dif < 4: #Mantenemos un margen de error de 4 grados.
            rele.off()
            actualizar_pantalla(oled, "normal", round(temp,2), round(dif,2), round(tideal,2), incendio)
            estado = 2
            
        if dif <= -4:
            
            rele.off()
            actualizar_pantalla(oled, "normal", round(temp,2), round(dif,2), round(tideal,2), incendio)
            if estado != 3:
                alexa_client.enviar_mensaje("¡Atencion!, temperatura muy baja detectada. Abriguese.")
            estado = 3
                
        if dif >= 4: #Mantenemos un margen de error de 3 grados.
            rele.on()
            actualizar_pantalla(oled, "ajuste", round(temp,2), round(dif,2), round(tideal,2), incendio)
            if estado != 4:
                alexa_client.enviar_mensaje("¡Atencion!, temperatura elevada, encendiendo sistema de refrigeración.")
            estado = 4
            
    if incendio:
        
        rele.off() 
        actualizar_pantalla(oled, "emergencia",round(temp,2), round(dif,2), round(tideal,2), incendio)
        alexa_client.enviar_mensaje("¡Atencion!, Incendio detectado. ¡Evacúe el área inmediatamente!")
        sonido_luces_alarma(duracion=15, delay_luces=10) #Encendemos luces y alarma.
        led.clear()
        estado = 5
        
    iteracion += 1
    print("-------------------------------------------------------------------------------------")
    print("\n")
    time.sleep(5)    
    
alexa_client.enviar_mensaje("¡Atencion!, Deteniendo Funcionamiento del sistema I.O.T")
alexa_client.cerrar() 
pynq_client.cerrar()
oled.clear_display()    
led.clear()
rele.off()

[PYNQ] Conectado al broker (127.0.0.1), rc= 0
[ALEXA] Intentando conectar al broker MQTT en 127.0.0.1:1883...
[ALEXA] Esperando conexión al broker...
[ALEXA] Conectado al broker MQTT correctamente.
[ALEXA] Cliente MQTT conectado correctamente.
-------------------------------------------------------------------------------------
Iteración de lectura y envio de datos número: 1
[PYNQ] Publicado en 'datos_sensores': {'temp': 25.380390167236328, 'part': 0, 'iteracion': 1}
[PYNQ] Esperando respuesta para iteración 1...
[PYNQ] Mensaje en datos_modelos: {"tideal": 23.78, "incendio": false, "iteracion": 1}
[PYNQ] Respuesta procesada: {'tideal': 23.78, 'incendio': False, 'iteracion': 1}
Valores Enviados de los Sensores: Temperatura=25.380390167236328, Particulas=0
Valores Recibidos por el Modelo: tideal=23.78, incendio=False, iteración1
-------------------------------------------------------------------------------------


